# #2 Flatten the Stack

In [2]:
import polars as pl

In [27]:
orders = (
    pl.read_csv(
        "sales_orders.csv",
        try_parse_dates=True
    )
)

dtype = pl.List(
    pl.Struct([
        pl.Field(
            "product",
            pl.Struct([
                pl.Field("product_name", pl.String),
                pl.Field("product_price", pl.Float64),
            ])
        ),
        pl.Field("quantity", pl.Int64),
    ])
)

sales_orders = (
    orders
    .with_columns(pl.col("line_items").str.json_decode(dtype))
    .explode("line_items")
    .unnest("line_items")
    .unnest("product")
    .with_columns((pl.col("product_price") * pl.col("quantity")).alias("sales"))
)

sales_orders

order_number,order_date,product_name,product_price,quantity,fulfillment,sales
i64,date,str,f64,i64,str,f64
387005,2016-01-22,"""MGS Age of Empires II Gold Edi…",32.0,3,"""In store""",96.0
387005,2016-01-22,"""A. Datum Bridge Digital Camera…",186.9,2,"""In store""",373.8
387005,2016-01-22,"""WWI Desktop PC1.80 E1801 Silve…",269.9,1,"""In store""",269.9
395005,2016-01-30,"""Contoso DVD 60 DVD Storage Bin…",22.89,5,"""In store""",114.45
395005,2016-01-30,"""SV DVD 38 DVD Storage Binder E…",9.99,5,"""In store""",49.95
…,…,…,…,…,…,…
2175000,2020-12-14,"""The Phone Company Smart phones…",129.0,2,"""In store""",258.0
2175000,2020-12-14,"""The Phone Company PDA Phone Un…",238.0,4,"""In store""",952.0
2175000,2020-12-14,"""Fabrikam Laptop10.1 M0101 Silv…",384.9,3,"""In store""",1154.7


In [30]:
sales_orders.filter(pl.col("fulfillment") == "Online").select("sales").sum()

sales
f64
18238.98
